In [1]:
### Load necessary modules -------------------------------
# interactive plotting
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # ‘png’, ‘retina’, ‘jpeg’, ‘svg’, ‘pdf’

# plotting libraries
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
sns.set_theme()

# Data management libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import os
from collections import Counter

# Machine learning libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from scipy.stats import norm
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from neuralsens import partial_derivatives as ns

#Clustering
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.vq import vq
from sklearn.metrics import silhouette_score
import skfuzzy as fuzz
from skfuzzy import cmeans
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import pairwise_distances
from itertools import combinations
from scipy.spatial.distance import cdist

#Profiling
import line_profiler
from line_profiler import LineProfiler
%load_ext line_profiler

from funciones_limpieza import *
from funciones_clustering import *
from utils_aprendizaje_no_supervisado import * 


# Preprocesado y cálculo PCA datos parque Myrtle

In [2]:
carpeta = '/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Datos_TFMGamesa/Datos Parque MYRTLE/Plots Myrtle'
pca, X_pca = preprocesado_y_pca(carpeta, 10)

/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Clustering/funciones_limpieza.py:124: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  plots = pd.read_csv(nombre_csv, index_col = False, sep = ";")
/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Clustering/funciones_limpieza.py:124: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  plots = pd.read_csv(nombre_csv, index_col = False, sep = ";")
/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Clustering/funciones_limpieza.py:124: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  plots = pd.read_csv(nombre_csv, index_col = False, sep = ";")
/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Clustering/funciones_limpieza.py:124: ParserWarning: Length of header or names does not match len

In [3]:
#Nos quedamos con las tres primeras componentes principales pues vimos que eran las más representativas
X_pca_3 = X_pca[:, :3]
X_pca_3.size

9071790

# C-means

In [ ]:
def dunn_index(X, centroids, u):
    """
    Cálculo del índice de Dunn para evaluar la calidad del clustering.
    """
    # Distancia mínima entre dos centros de clusters
    inter_cluster_dist = np.min(cdist(centroids, centroids))
    
    # Máxima distancia intra-cluster entre puntos y sus centros
    intra_cluster_dist = 0
    for i in range(len(centroids)):
        cluster_points = X[np.argmax(u == i, axis=0)]  # Puntos del cluster
        intra_cluster_dist = max(intra_cluster_dist, np.max(cdist(cluster_points, [centroids[i]])))
    
    return inter_cluster_dist / intra_cluster_dist


In [ ]:
def fuzzy_cmeans_cross_validation(X, n_clusters=2, m_values=[1.5, 2.0, 2.5], error_values=[0.001, 0.005, 0.01], maxiter_values=[100, 200, 500]):
    """
    Validación cruzada para optimizar los parámetros de Fuzzy C-Means y graficar el índice de Dunn.

    Parámetros:
    -----------
    - X: Matriz de datos (tamaño: n_samples x n_features)
    - n_clusters: Número de clusters (fijo en 2 para este caso)
    - m_values: Lista de valores para el parámetro de difusividad
    - error_values: Lista de valores para el parámetro de tolerancia de error
    - maxiter_values: Lista de valores para el número máximo de iteraciones

    Devuelve:
    --------
    - El conjunto de parámetros con el mejor índice de Dunn
    """
    
    # Preparar el KFold para la validación cruzada
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Inicializar la mejor métrica
    best_dunn = -np.inf
    best_params = None
    
    # Para almacenar los resultados de Dunn
    results = []
    
    # Hacer un barrido sobre todos los parámetros posibles
    for m in m_values:
        for error in error_values:
            for maxiter in maxiter_values:
                print(f"Evaluando parámetros: m={m}, error={error}, maxiter={maxiter}")
                
                # Listar los Dunns para cada fold
                dunn_scores = []
                
                # Realizar la validación cruzada
                for train_idx, test_idx in kf.split(X):
                    X_train, X_test = X[train_idx], X[test_idx]
                    
                    # Aplicar Fuzzy C-Means a los datos de entrenamiento
                    cntr, u, u0, d, jm, p, fpc = cmeans(
                        X_train.T,  # Datos transpuestos
                        c=n_clusters,  # Número de clusters
                        m=m,  # Difusividad
                        error=error,  # Tolerancia de error
                        maxiter=maxiter,  # Máximas iteraciones
                        init=None,  # Inicialización automática
                        seed=42  # Semilla para reproducibilidad
                    )
                    
                    # Obtener los centroides
                    centroids = cntr
                    
                    # Calcular el índice de Dunn para este pliegue
                    dunn_score = dunn_index(X_test, centroids, u)
                    dunn_scores.append(dunn_score)
                
                # Promediar los resultados de la validación cruzada
                mean_dunn = np.mean(dunn_scores)
                print(f"Índice de Dunn promedio para m={m}, error={error}, maxiter={maxiter}: {mean_dunn}")
                
                # Guardar los resultados para el gráfico
                results.append((m, error, maxiter, mean_dunn))
                
                # Si encontramos un mejor índice de Dunn, guardamos los parámetros
                if mean_dunn > best_dunn:
                    best_dunn = mean_dunn
                    best_params = (m, error, maxiter)
    
    # Convertir los resultados a un array de numpy para facilidad de uso
    results = np.array(results)
    
    # Crear el gráfico
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Graficar el índice de Dunn para cada combinación de parámetros
    scatter = ax.scatter(results[:, 0], results[:, 1], c=results[:, 3], cmap='viridis', s=100, edgecolors='black')
    ax.set_xlabel('m (Coef. de Difusividad)')
    ax.set_ylabel('Error')
    ax.set_title('Evolución del Índice de Dunn con los Parámetros')

    # Añadir barra de color para mostrar los valores del índice de Dunn
    cbar = plt.colorbar(scatter)
    cbar.set_label('Índice de Dunn')
    
    # Mostrar el gráfico
    plt.tight_layout()
    plt.show()
    
    print(f"Mejores parámetros encontrados: m={best_params[0]}, error={best_params[1]}, maxiter={best_params[2]}")
    return best_params


In [6]:
#fuzzy_cmeans_cross_validation(X_pca_3)

#### ENTRENO CMEANS CON 2 CLUSTERS

In [4]:
# Transponer X_pca porque skfuzzy espera datos como (n_features, n_samples)
X_pca_3_T = X_pca_3.T

# Aplicar Fuzzy C-Means con 2 clusters
n_clusters = 2
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    X_pca_3_T,           # Datos (transpuestos)
    c=n_clusters,      # Número de clusters
    m=2.0,             # Coef. de difusividad (mayor → más difuso)
    error=0.005,       # Tolerancia de error (Criterio de parada: cambio mínimo en la matriz de pertenencias)
    maxiter=1000,      # Iteraciones máximas
    init=None,         # Inicialización automática de pertenencias (u0)
    seed=42            # Para reproducibilidad
)

#BREVE EXPLICACIÓN
#-----------------
#Fuzzy C-Means es un algoritmo de clustering difuso, lo que significa que cada punto puede pertenecer parcialmente a varios clusters. A 
#diferencia de K-Means (clustering duro), donde un punto pertenece a un solo cluster, FCM asigna un grado de pertenencia a cada punto con 
#respecto a cada cluster.

#¿Cómo funciona el algoritmo?
#1. Inicialización
#Se elige el número de clusters c (por ejemplo, n_clusters = 2). 
#Se inicializa aleatoriamente una matriz de pertenencia u (de tamaño c × n) donde u[i, j] es el grado de pertenencia del punto j al cluster i.
#El parámetro m > 1 (normalmente m=2) controla cuán difusa es esa pertenencia.

#2. Iteración
#El algoritmo sigue este ciclo:
#a) Actualizar centroides (centros de clusters): Cada centroide v_i se actualiza como un promedio ponderado de los puntos, ponderado por los
#grados de pertenencia elevados a m

#b) Actualizar matriz de pertenencia u: Cada u_{ij} se calcula en función de la distancia entre el punto x_j y el centro v_i
#Esto asegura que:
#Si un punto está más cerca de un centroide, tendrá una mayor pertenencia a ese cluster.
#Las pertenencias de un punto a todos los clusters suman 1.

#c) Criterio de parada:
#Se mide el cambio en la matriz de pertenencia entre iteraciones sucesivas. Si ese cambio es menor que el parámetro error, o se alcanzan 
#maxiter iteraciones, se detiene el algoritmo.


#EXPLICACIÓN PARÁMETROS
#----------------------
#- X_pca_T:	Matriz de datos transpuesta: filas = características, columnas = puntos
#- c: Número de clusters a encontrar
#- m: Fuzziness exponent: mide el grado de superposición entre clusters.
#          Si m → 1, el modelo se parece a K-Means (menos difuso)
#          Si m > 1, los clusters son más difusos (común usar m = 2)
#- error: Umbral de convergencia: si el cambio entre iteraciones es menor, se detiene
#- maxiter: Número máximo de iteraciones que permite
#- init: Matriz de pertenencias inicial (puede dejarse como None)
#- seed: Fija la aleatoriedad de la inicialización para que el resultado sea reproducible

#EXPLICACIÓN SALIDA
#------------------
#- cntr: Matriz con los centroides difusos de los clusters (shape: c x features)
#- u: Matriz final de pertenencias (shape: c x n_samples)
#- u0: Matriz de pertenencias iniciales (antes de empezar a iterar)
#- d: Distancia de cada punto a cada centro (usada internamente)
#- jm: Evolución del coste (función objetivo) en cada iteración
#- p: Número de iteraciones hasta converger
#- fpc: Fuzzy Partition Coefficient (entre 0 y 1). Cuanto más cercano a 1, mejor la partición (mide cuán claras son las asignaciones de pertenencia)

In [8]:
#Vemos la calidad del clustering con FPC (Fuzzy Partition Coefficient). Es un valor entre 0 y 1:
#- Cercano a 1: buena partición, los puntos tienen pertenencias claras (alta certeza).
#- Cercano a 0: partición difusa, los puntos están muy mezclados entre clusters.
print(f"El índice FPC para nuestro modelo de C-means con dos clusters es: {fpc}")

El índice FPC para nuestro modelo de C-means con dos clusters es: 0.838147808868499


In [9]:
dunn_index(X_pca_3, cntr, u) #nuestros centroides y clusters están muy juntos (se puede ver en la grafica PC2 vs PC3)

0.0

In [10]:
#Obtenemos las etiquetas de los cluster con el mayor grado de pertenencia para cada cluster (esto sería como el resultado obtenido con kmeans)

# Asignar cada punto al cluster con mayor grado de pertenencia
cluster_labels_cmeans = np.argmax(u, axis=0)

# Contar elementos por cluster
unique, counts = np.unique(cluster_labels_cmeans, return_counts=True)

# Calcular porcentaje
total_samples = len(cluster_labels_cmeans)
percentages = (counts / total_samples) * 100

# Crear DataFrame con resultados
df_clusters_cmeans = pd.DataFrame({
    '# Samples': counts,
    'Percentage (%)': percentages
}, index=unique)

# Mostrar resultados
print('Distribución de muestras por cluster (C-Means):')
print(df_clusters_cmeans)


#Tenemos más puntos en el cluster 0 que en el 1 (número de elementos en los clusters similares a Kmeans pero en este caso tenemos algo más de 
#elementos en el cluster 1 esto puede deberse a puntos con grados de pertenencia similares en los dos clusters)

Distribución de muestras por cluster (C-Means):
   # Samples  Percentage (%)
0    1897981       62.765375
1    1125949       37.234625


#### VISUALIZACIÓN CLUSTERS

In [11]:
# Obtener etiquetas predichas (máxima pertenencia)
cluster_labels = np.argmax(u, axis=0)

# Obtener los centros de los clusters desde C-means
cluster_centers = cntr  # asumimos que esta variable contiene los centroides

# Muestrear un pequeño porcentaje de los datos para visualizar
sample_size = int(X_pca_3.shape[0] * 1)  # 100% en este caso
sampled_indices = np.random.choice(X_pca_3.shape[0], sample_size, replace=False)
sampled_X = X_pca_3[sampled_indices, :]
sampled_labels = cluster_labels[sampled_indices]

# Número de componentes principales
n_components = X_pca_3.shape[1]
pairs = list(combinations(range(n_components), 2))

# Crear subgráficos
fig, axes = plt.subplots(len(pairs), 1, figsize=(6, len(pairs) * 3))
if len(pairs) == 1:
    axes = [axes]

# Visualización por pares de componentes principales
for idx, (i, j) in enumerate(pairs):
    ax = axes[idx]

    # Scatter por clúster para añadir leyenda
    for cluster_id in np.unique(sampled_labels):
        cluster_points = sampled_X[sampled_labels == cluster_id]
        ax.scatter(
            cluster_points[:, i], cluster_points[:, j],
            label=f"Cluster {cluster_id}",
            alpha=0.5,
            s=10
        )

    # Visualización de los centros
    ax.scatter(
        cluster_centers[:, i], cluster_centers[:, j],
        c='red', marker='X', s=80, label="Centroides"
    )

    # Ajustar ejes al rango de los puntos
    ax.set_xlim([sampled_X[:, i].min() - 0.5, sampled_X[:, i].max() + 0.5])
    ax.set_ylim([sampled_X[:, j].min() - 0.5, sampled_X[:, j].max() + 0.5])

    ax.set_title(f"C-means: PC{i+1} vs PC{j+1}")
    ax.set_xlabel(f"PC{i+1}")
    ax.set_ylabel(f"PC{j+1}")
    ax.legend(loc='upper right', fontsize=8)

# Ajustar disposición y guardar
plt.tight_layout()
plt.savefig('/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Clustering/Imagenes/CMeans/clusters_cmeans_k2_PCA3.png')
plt.close()


#### VALOR MEDIO PCA DE LOS CENTROIDES

In [16]:
# Saber el valor medio que tienen los centroides de un cluster nos dará una idea intuitiva de los valores que toman para cada PCA los 
# puntos de un cluster

# Print values of the cluster centers
n_clusters = 2
print(cluster_centers) #los centroides están en el espacio de PCA

# Plot the values of the cluster center
for i in range(cluster_centers.shape[1]):
    plt.figure(figsize=(5, 3))
    sns.barplot(x=range(n_clusters), y=cluster_centers[:,i], edgecolor='black', palette='viridis')
    plt.xticks(range(n_clusters))
    plt.xlabel('Cluster')
    plt.ylabel('PC{}'.format(i+1))
    plt.title('Cluster centers for PC{}'.format(i+1))

    # Guardar la imagen en lugar de mostrarla
    # Aquí puedes cambiar la ruta y el nombre del archivo según lo desees
    nombre_archivo = f'/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Clustering/Imagenes/Cmeans/Cmeans2_cluster_centers_PC{i+1}.png'  # Nombre del archivo de la imagen
    plt.savefig(nombre_archivo, dpi=300)  # Guarda la imagen con alta resolución
    plt.close()  # Cierra la figura después de guardarla para evitar que se acumulen múltiples gráficos

[[-3.57746074  0.29083528  0.21206906]
 [ 5.86637901 -0.20482372  0.11343113]]


/var/folders/wn/gxtmyy811gg8d_h61xk9hfkm0000gn/T/ipykernel_3662/1635054801.py:11: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=range(n_clusters), y=cluster_centers[:,i], edgecolor='black', palette='viridis')
/var/folders/wn/gxtmyy811gg8d_h61xk9hfkm0000gn/T/ipykernel_3662/1635054801.py:11: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=range(n_clusters), y=cluster_centers[:,i], edgecolor='black', palette='viridis')
/var/folders/wn/gxtmyy811gg8d_h61xk9hfkm0000gn/T/ipykernel_3662/1635054801.py:11: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barpl

#### GRADOS DE PERTENENCIA CLUSTERS

In [17]:
#Vamos a crear un histograma en el que podamos ver para cada cluster la cantidad de puntos que pertenecen a ese cluster con cierto grado de 
#pertenencia. Si hay muchos puntos en grados de pertenencia cercanos a 1 indica que los puntos se han asignado con claridad ese cluster.

# Crear una figura para todos los histogramas
fig, axes = plt.subplots(n_clusters, 1, figsize=(8, 4 * n_clusters))

# Si solo hay un clúster, convertir axes en lista para que sea iterable
if n_clusters == 1:
    axes = [axes]

for i in range(n_clusters):
    # Seleccionar grados de pertenencia de los puntos asignados al cluster i
    pertenencias = u[i, cluster_labels == i]

    ax = axes[i]
    ax.hist(pertenencias, bins=20, color='skyblue', edgecolor='black')
    ax.set_title(f'Grados de pertenencia para el Clúster {i}')
    ax.set_xlabel('Grado de pertenencia')
    ax.set_ylabel('Cantidad de puntos')
    ax.grid(True)

plt.tight_layout()

# Guardar imagen (puedes cambiar el nombre dinámicamente si quieres)
plt.savefig('/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Clustering/Imagenes/CMeans/GradosPertenencia_cmeans_k2_PCA3.png')

plt.close()

#Vemos que la mayoría de puntos pertenecen a cada cluster con grados de pertenencia cercanos a uno por lo que los clusters son bastante 
#robustos. Los puntos con mennores grados de pertenencia pueden ser puntos situados en las fronteras de los clusters

### Ahora probamos este clustering con los datos de inversores con un funcionamiento a priori normal y uno con comportamiento anómalo

In [18]:
#Vamos a considerar el inversor 34 y el 11 del día 13-02-25. Sabemos que el inversor 11 tuvo fallo de explosión tras congelar

#### INVERSOR 11

In [5]:
#LECTURA Y PPREPROCESADO INVERSOR 11
#-----------------------------------

carpeta_explosion_congelar = '/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Datos_TFMGamesa/FalloExplosionTrasCongelarPruebas 13_02_25/inv11'
X_pca_explosion_congelar = aplicar_pca_datos(carpeta_explosion_congelar, pca)
# Seleccionar las primeras 3 componentes principales
X_pca_3_explosion_congelar = X_pca_explosion_congelar[:, :3]
X_pca_3_explosion_congelar.size

/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Clustering/funciones_limpieza.py:124: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  plots = pd.read_csv(nombre_csv, index_col = False, sep = ";")


129573

In [6]:
# Asignar a cada punto el cluster con mayor pertenencia

# Transponer los nuevos datos igual que los originales
X_pca_3_explosion_congelar_T = X_pca_3_explosion_congelar.T

# Predecir los grados de pertenencia para los nuevos datos
u_explosion_congelar, _, _, _, _, _ = fuzz.cluster.cmeans_predict(
    test_data = X_pca_3_explosion_congelar_T,
    cntr_trained = cntr,
    m         = 2.0,
    error     = 0.005,
    maxiter   = 1000
)
clusters_predichos_explosion_congelar = np.argmax(u_explosion_congelar, axis=0)

# Número total de elementos
total_elements = clusters_predichos_explosion_congelar.shape[0]

# Contar elementos por clúster
cluster_counts = np.bincount(clusters_predichos_explosion_congelar)

# Mostrar resultados
for cluster_id, count in enumerate(cluster_counts):
    percentage = (count / total_elements) * 100
    print(f"Cluster {cluster_id}: {count} elementos ({percentage:.2f}%)")

Cluster 0: 28271 elementos (65.46%)
Cluster 1: 14920 elementos (34.54%)


In [15]:
# Obtener etiquetas predichas por C-means (usamos los grados de pertenencia y asignamos el cluster con mayor pertenencia)
clusters_predichos_cmeans = np.argmax(u_explosion_congelar, axis=0)

# Obtener los centros de los clusters (esto es cntr en C-means)
cluster_centers_cmeans = cntr

# Muestrear un pequeño porcentaje de los datos para visualizar
sample_size = int(X_pca_3_explosion_congelar.shape[0] * 1)  # 100% en este caso, puedes ajustar
sampled_indices = np.random.choice(X_pca_3_explosion_congelar.shape[0], sample_size, replace=False)
sampled_X = X_pca_3_explosion_congelar[sampled_indices, :]
sampled_labels = clusters_predichos_cmeans[sampled_indices]

# Número de componentes principales
n_components = X_pca_3_explosion_congelar.shape[1]
pairs = list(combinations(range(n_components), 2))

# Crear los subgráficos
fig, axes = plt.subplots(len(pairs), 1, figsize=(6, len(pairs) * 3))
if len(pairs) == 1:
    axes = [axes]

# Visualización por pares de componentes principales
for idx, (i, j) in enumerate(pairs):
    ax = axes[idx]

    # Creamos un scatter plot para cada cluster individual para poder añadir la leyenda
    for cluster_id in np.unique(sampled_labels):
        cluster_points = sampled_X[sampled_labels == cluster_id]
        ax.scatter(
            cluster_points[:, i], cluster_points[:, j],
            label=f"Cluster {cluster_id}",
            alpha=0.5,
            s=10
        )

    # Visualización de los centros (centroides de C-means)
    ax.scatter(
        cluster_centers_cmeans[:, i], cluster_centers_cmeans[:, j],
        c='red', marker='X', s=80, label="Centroides"
    )

    ax.set_xlim([sampled_X[:, i].min() - 1, sampled_X[:, i].max() + 1])
    ax.set_ylim([sampled_X[:, j].min() - 1, sampled_X[:, j].max() + 1])

    ax.set_title(f"PC{i+1} vs PC{j+1}")
    ax.set_xlabel(f"PC{i+1}")
    ax.set_ylabel(f"PC{j+1}")
    ax.legend(loc='upper right', fontsize=8)

# Ajustar disposición y guardar
plt.tight_layout()
plt.savefig('/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Clustering/Imagenes/CMeans/DistribucionClusters_CMeans2_inv11Dia130225.png')
plt.close()


#### INVERSOR 34

In [8]:
#LECTURA Y PPREPROCESADO INVERSOR 34
#-----------------------------------

carpeta_inv34_130225 = '/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Datos_TFMGamesa/FalloExplosionTrasCongelarPruebas 13_02_25/inv34'  
X_pca_inv34_130225 = aplicar_pca_datos(carpeta_inv34_130225, pca)
# Seleccionar las primeras 3 componentes principales
X_pca_3_inv34_130225 = X_pca_inv34_130225[:, :3]
X_pca_3_inv34_130225.size

/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Clustering/funciones_limpieza.py:124: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  plots = pd.read_csv(nombre_csv, index_col = False, sep = ";")


129597

In [9]:
# Asignar a cada punto el cluster con mayor pertenencia

# Transponer los nuevos datos igual que los originales
X_pca_3_inv34_130225_T = X_pca_3_inv34_130225.T

# Predecir los grados de pertenencia para los nuevos datos
u_inv34_130225, _, _, _, _, _ = fuzz.cluster.cmeans_predict(
    test_data = X_pca_3_inv34_130225_T,
    cntr_trained = cntr,
    m         = 2.0,
    error     = 0.005,
    maxiter   = 1000
)
clusters_predichos_inv34_130225 = np.argmax(u_inv34_130225, axis=0)

# Número total de elementos
total_elements = clusters_predichos_inv34_130225.shape[0]

# Contar elementos por clúster
cluster_counts = np.bincount(clusters_predichos_inv34_130225)

# Mostrar resultados
for cluster_id, count in enumerate(cluster_counts):
    percentage = (count / total_elements) * 100
    print(f"Cluster {cluster_id}: {count} elementos ({percentage:.2f}%)")

Cluster 0: 26555 elementos (61.47%)
Cluster 1: 16644 elementos (38.53%)


In [14]:
# Obtener etiquetas predichas por C-means (usamos los grados de pertenencia y asignamos el cluster con mayor pertenencia)
clusters_predichos_cmeans = np.argmax(u_inv34_130225, axis=0)

# Obtener los centros de los clusters (esto es cntr en C-means)
cluster_centers_cmeans = cntr

# Muestrear un pequeño porcentaje de los datos para visualizar
sample_size = int(X_pca_3_inv34_130225.shape[0] * 1)  # 100% en este caso, puedes ajustar
sampled_indices = np.random.choice(X_pca_3_inv34_130225.shape[0], sample_size, replace=False)
sampled_X = X_pca_3_inv34_130225[sampled_indices, :]
sampled_labels = clusters_predichos_cmeans[sampled_indices]

# Número de componentes principales
n_components = X_pca_3_inv34_130225.shape[1]
pairs = list(combinations(range(n_components), 2))

# Crear los subgráficos
fig, axes = plt.subplots(len(pairs), 1, figsize=(6, len(pairs) * 3))
if len(pairs) == 1:
    axes = [axes]

# Visualización por pares de componentes principales
for idx, (i, j) in enumerate(pairs):
    ax = axes[idx]

    # Creamos un scatter plot para cada cluster individual para poder añadir la leyenda
    for cluster_id in np.unique(sampled_labels):
        cluster_points = sampled_X[sampled_labels == cluster_id]
        ax.scatter(
            cluster_points[:, i], cluster_points[:, j],
            label=f"Cluster {cluster_id}",
            alpha=0.5,
            s=10
        )

    # Visualización de los centros (centroides de C-means)
    ax.scatter(
        cluster_centers_cmeans[:, i], cluster_centers_cmeans[:, j],
        c='red', marker='X', s=80, label="Centroides"
    )

    ax.set_xlim([sampled_X[:, i].min() - 1, sampled_X[:, i].max() + 1])
    ax.set_ylim([sampled_X[:, j].min() - 1, sampled_X[:, j].max() + 1])

    ax.set_title(f"PC{i+1} vs PC{j+1}")
    ax.set_xlabel(f"PC{i+1}")
    ax.set_ylabel(f"PC{j+1}")
    ax.legend(loc='upper right', fontsize=8)

# Ajustar disposición y guardar
plt.tight_layout()
plt.savefig('/Users/lucialopez/Documents/MBD/Segundo_cuatrimestre/TFM/Clustering/Imagenes/CMeans/DistribucionClusters_CMeans2_inv34Dia130225.png')
plt.close()


In [11]:
print(X_pca_3_explosion_congelar.shape[0])
print(X_pca_3_inv34_130225.shape[0])


3
3
